## Testing installed packages via Conda 

We will first ensure that the installed packages are working properly. You should be able to run the following cells. Once everything passes, continue onto the geometry assignment. 

In [227]:
try: 
    import meshcat
    import meshcat.geometry as geom
    import meshcat.transformations as tf
    import numpy as np
    import time
    import numpy as np
    import jax.numpy as jnp
    import jax
    from jaxlie import SE2, SE3
    import matplotlib.pyplot as plt
    
    print('Import packages works! Great work following directions :D !')
except Exception as e:
    print('Something went wrong. The following error tells you what happened. Go through README.md again and see what went wrong')
    print(e)

Import packages works! Great work following directions :D !


## Meshcat Visualization Tool
Meshcat is a WebGL based 3D renderer that works on your web browser and is based in javascript. We will be using this visualization tool to plot reference frames, perform transforms, and build robot visualizations. Run the code below to familiarize yourself with `meshcat-python` which is a wrapper around meshcat in python. You can check out more examples at the [github page](https://github.com/meshcat-dev/meshcat-python/tree/master/examples) . 


In [228]:
# Create a new visualizer
vis = meshcat.Visualizer()

# Create a jupyter cell that renders the visalizer by calling the function below
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


### Reference frame warmup
Here we will create rotation matrices and transform them according to SE(3) transforms that we will generate. You can use the `meshcat.transformations` package to create transforms or `jaxlie.SE3` to create transformation matrices. 

#### With meshcat

In [229]:
# the brackets creates an instance of an object and starts the creation of a tree structure 
frame = vis['free frame']
# you can set object properties like geometry, color, text, etc, using the meshcat.geometry package (aliased as geom)
frame.set_object(geom.triad())
# you can also transforms the created object relative to its orgin frame using the meshcat.transformations package (aliased as tf)
# first let's translate vertically by 1 (meshcat generates transforms through numpy)
T_WA = tf.translation_matrix(np.array([0., 0., 1.0])) 
print('pure translation \n',T_WA)
# then perform a random rotation relative to A
T_AB = tf.random_rotation_matrix()
print('pure rotation \n', T_AB)
# the operation to set the transform on the frame we created is below using the python matrix @ operator
frame.set_transform(T_WA@T_AB)

pure translation 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
pure rotation 
 [[-0.81505087 -0.42403684  0.39482254  0.        ]
 [-0.14709634  0.81056149  0.56687983  0.        ]
 [-0.56040588  0.40395895 -0.72302311  0.        ]
 [ 0.          0.          0.          1.        ]]


In [230]:
# order matters for elements of SE(3), try flipping the order and see what happens
# let's create a new frame to see this in action!
frame2 = vis['free frame2']
# you can set object properties like geometry, color, text, etc, using the meshcat.geometry package (aliased as geom)
frame2.set_object(geom.triad(scale=0.5))
frame2.set_transform(T_AB@T_WA)
# notice above that a new coordinate system is created in a random location (since we rotate first, then translate)

#### With jaxlie

In [231]:
vis.delete()
# the brackets creates an instance of an object and starts the creation of a tree structure 
frame = vis['free frame']
# you can set object properties like geometry, color, text, etc, using the meshcat.geometry package (aliased as geom)
frame.set_object(geom.triad())
# you can also transforms the created object relative to its orgin frame using the meshcat.transformations package (aliased as tf)
# first let's translate vertically by 1 (meshcat generates transforms through numpy)
T_WA = SE3.exp(np.array([0., 0., 1.0, 0., 0., 0.])).as_matrix()
print('pure translation \n',T_WA)
# then perform a random rotation relative to A
T_AB = SE3.exp(
    np.concatenate([np.zeros(3), np.random.normal(size=(3,)),])
).as_matrix()
print('pure rotation \n', T_AB)
# the operation to set the transform on the frame we created is below using the python matrix @ operator
# you will need to transform the matrix to a float64 instead of a 32 bit number using jax
frame.set_transform(np.array(T_WA@T_AB, dtype=np.float64))

pure translation 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
pure rotation 
 [[-0.44009984 -0.19439209  0.8766552   0.        ]
 [ 0.6505558  -0.74196494  0.16206746  0.        ]
 [ 0.6189425   0.6416388   0.45300102  0.        ]
 [ 0.          0.          0.          1.        ]]


### Basic Geometry
You can also add multiple geometry elements to objects including frames. Below are a few basic operations you may find useful.

In [232]:
# let us first delete the objects in the visualization. note that this object deletes all elements that are created from the original vis object.
vis.delete()

# we will create two bodies and attach an axis and a geometry element
body1 = vis["body1"]
body1["geom"].set_object(
    geom.Box([0.5, 0.3, 0.2])
)
body1["axis"].set_object(geom.triad())
body1.set_transform(tf.translation_matrix(np.array([0.,0.,1.0])))

body2 = vis["body2"]
body2["geom"].set_object(
    geom.Cylinder(0.5,0.1)
)
body2["axis"].set_object(geom.triad())
body2.set_transform(tf.random_rotation_matrix())

# note it is possible to also translate the elements attached to the body object
body2["axis"].set_transform(tf.translation_matrix(np.array([0.,0.25,.0])))
# the operation above sets the axis label on the end of the cylinder

### A Cart-Pole Robot Example
<img src="cart_pend.png" alt="image" width="15%" height="auto">

The cart pole has two degrees of freedom -- $y, \theta$ which define the cart left/right movement and the pole rotation about the cart.

In [233]:
vis.delete()
cart_pole = vis["cart_pole"]
# delete just in case there are anything extra that carries over
cart_pole.delete()
cart = cart_pole["cart"]
pivot = cart["pivot"]
pole = pivot["pole"]
cart.set_object(geom.Box([0.3, 0.5, 0.2]))
pole.set_object(geom.Box([0.05, 0.05, 1.0]))
pole.set_transform(tf.translation_matrix([0, 0, 0.5]))

Cell below renders the visualization closer

In [234]:
vis.jupyter_cell()

### Animate the Cart-Pole
Here, we animate the cart y position with $y(t) = \sin(t)$ and pole is rotated $\theta(t) = t/4$ for $t=[0,4]$ with $dt = 0.1$. Note that the movements are prescribed and purely kinematic and DO NOT abide by any laws of physics.

In [235]:
# create a for-loop through time
for t in np.arange(0, 4, step=0.1):
    # set the y-pos transform through the tf/jaxlie module on the cart coordinate system
    cart.set_transform(tf.translation_matrix([0,np.sin(t), 0]))
    # set the theta-angle transform through the tf/jaxlie module on the pole pivit coordinate system
    pivot.set_transform(tf.rotation_matrix(t / 4, [1, 0, 0]))
    # the time module creates a real-time effect for visualization 
    time.sleep(0.1)

## Assignment: Visualize and render the following robots
Visualizer the following 2D robot using SE(3) transforms and animate the joints accordingly. 

### Q 1: Double Pendulum
<img src="double_pendulum.png" alt="image" width="15%" height="auto">

The double pendulum consists of two degrees of freedom $\theta_1, \theta_2$ which are the pole rotation points at the base and at the second link measured **relative** to the parent frame. 
Rotate the base joint from $\theta_1(t) = 2 \sin(t)$ and the second joint $\theta_2(t) = -\sin(t)$ for $t\in[0,10s]$ in increments of $dt = 0.1$

In [ ]:
## RUN ME SO THAT YOU CLEAR THE PREVIOUS EXAMPLES AND CREATE A WINDOW CLOSER TO YOUR CODE
vis.delete()
vis.jupyter_cell()

### ADD CODE BELOW

In [278]:
## ADD CODE HERE THAT CONSTRUCTS THE DOUBLE PENDULUM
def build_leg(origin, parent_frame, num=0):
    leg_1 = parent_frame[f"leg_{num}_1"]
    leg_1["geom"].set_object(
        geom.Cylinder(0.3,0.03)
    )
    leg_1["axis"].set_object(geom.triad())
    leg_1["axis"].set_transform(tf.translation_matrix(np.array([0.,-0.15,0])) @ tf.rotation_matrix(np.pi/2, [1, 0, 0]))
    leg_1.set_transform(origin @ tf.rotation_matrix(np.pi + .8, [1, 0, 0]) @ tf.translation_matrix(np.array([0.,0.15,0.])))

    leg_2 = leg_1["leg_2"]
    leg_2["geom"].set_object(
        geom.Cylinder(0.2,0.01)
    )
    leg_2["axis"].set_object(geom.triad())
    leg_2["axis"].set_transform(tf.translation_matrix(np.array([0.,-0.1,0])) @ tf.rotation_matrix(np.pi/2, [1, 0, 0]))
    leg_2.set_transform(tf.translation_matrix(np.array([0.,0.15,0])) @ tf.rotation_matrix(1.2, [1, 0, 0]) @ tf.translation_matrix(np.array([0.,0.1,0])))
    return leg_1, leg_2

leg_1, leg_2 = build_leg(tf.translation_matrix(np.array([0,0,0])), vis)

In [315]:
def set_leg_position(curr_leg_1, curr_leg_2, leg_1_offset, origin, leg_2_offset):
    curr_leg_1.set_transform(origin @ tf.rotation_matrix(np.pi + .8 + leg_1_offset, [1, 0, 0]) @ tf.translation_matrix(np.array([0.,0.15,0.])))
    curr_leg_2.set_transform(tf.translation_matrix(np.array([0.,0.15,0])) @ tf.rotation_matrix(1.2 + leg_2_offset, [1, 0, 0]) @ tf.translation_matrix(np.array([0.,0.1,0])))
    

### ANIMATE CODE BELOW

In [269]:
## ADD CODE HERE THAT ANIMATES THE JOINTS OF THE DOUBLE PENDULUM, USE THE CART PENDULUM EXAMPLE AS REFERENCE
DT = 0.1
TIME_LENGTH = 10
for delta in range(int(TIME_LENGTH / DT)):
    curr_time = delta*DT
    set_leg_position(leg_1, leg_2, 2 * np.sin(curr_time), -np.sin(curr_time))
    time.sleep(DT)

### A 2D Robot Dog
<img src="2d_dog.png" alt="image" width="20%" height="auto">

The robot dog has 5 degrees of freedom (can have more if you want, but for this assignment only 5 are considered). The 5 degrees of freedom consist of $y, \theta_{FH}, \theta_{FC}, \theta_{BH}, \theta_{BC}$ which are the $y$ base torso position relative to the world, the front hip and calf joints relative to the torso and parent frame, and the back hip and calf joints relative to the torso and parent frame.

Use cell below to create a jupyter cell to visualize your robot construction

In [305]:
## RUN ME SO THAT YOU CLEAR THE PREVIOUS EXAMPLES AND CREATE A WINDOW CLOSER TO YOUR CODE
vis.delete()
vis.jupyter_cell()

### ADD CODE BELOW

In [320]:
## ADD CODE HERE THAT CONSTRUCTS THE 2D ROBOT DOG


body = vis["body"]
body["geom"].set_object(
    geom.Box([0.1, 0.5, 0.2])
)
body.set_transform(tf.translation_matrix(np.array([0, 0, 0])))
body["axis"].set_object(geom.triad())
BACK_LEG_ORIGIN = tf.translation_matrix(np.array([0,-.25,0]))
FRONT_LEG_ORIGIN = tf.translation_matrix(np.array([0,.25,0]))
leg_1_1, leg_1_2 = build_leg(BACK_LEG_ORIGIN, body, num=1)
leg_2_1, leg_2_2 = build_leg(FRONT_LEG_ORIGIN, body, num=2)

## Animate the robot dog
Move the torso from $y(t) = 0.1 t$ in increments of $dt = 0.1$, rotate each joint $i$ by $\theta_i(t) = \sin(t)$ along the joint x-axis

### ANIMATE CODE BELOW

In [321]:
## RUN ME to CREATE A WINDOW CLOSER TO YOUR CODE
vis.jupyter_cell()

In [323]:
## ADD CODE HERE THAT ANIMATES THE JOINTS OF THE ROBOT DOG, USE THE CART PENDULUM EXAMPLE AS REFERENCE
DT = 0.1
TIME_LENGTH = 10

for delta in range(int(TIME_LENGTH / DT)):
    curr_time = delta*DT
    body.set_transform(tf.translation_matrix(np.array([0.,0.1 * curr_time,0.])))
    set_leg_position(leg_1_1, leg_1_2, np.sin(curr_time), BACK_LEG_ORIGIN, np.sin(curr_time))
    set_leg_position(leg_2_1, leg_2_2, np.sin(curr_time), FRONT_LEG_ORIGIN, np.sin(curr_time))
    time.sleep(DT)